### Read from data pool and filter

In [2]:
import glob
import shutil
import os
from tqdm import tqdm
data_pool_folder = "/home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/weapon_vp_crop/3ATDT/people"
save_dir = "/home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/all_images"
for path in tqdm(glob.glob(f'{data_pool_folder}/**/*.jpg', recursive=True)):
    name = path.split("/")[-1]
    month = path.split("/")[-3]
    day = path.split("/")[-2]
    # if "per" not in name and ("knife" in name or "pistol" in name):
    # new_name = f"{month}_{day}_{name}"
    new_path = os.path.join(save_dir, name)
    img = shutil.copy(path, new_path)

100%|██████████| 41678/41678 [08:07<00:00, 85.53it/s]  


In [1]:
import fastdup
from tqdm import tqdm
import shutil
import os
# a function to group connected components
def get_clusters(df, sort_by='count', min_count=2, ascending=False):
    # columns to aggregate
    agg_dict = {'filename': list, 'mean_distance': max, 'count': len}

    if 'label' in df.columns:
        agg_dict['label'] = list
    
    # filter by count
    df = df[df['count'] >= min_count]
    
    # group and aggregate columns
    grouped_df = df.groupby('component_id').agg(agg_dict)
    
    # sort
    grouped_df = grouped_df.sort_values(by=[sort_by], ascending=ascending)
    return grouped_df

def remove_dup_image(working_dir, trash_dir, clusters_df):
    all_images = set(os.listdir(working_dir))
    is_update = False
    component_groups = {}
    similar_images = set()
    for i, r in clusters_df.iterrows():
        group_name = i
        filenames = r['filename']
        component_groups[group_name] = filenames
        similar_images.update([path.split("/")[-1] for path in filenames])
    component_groups = {k:v for k, v in sorted(component_groups.items(), key=lambda x : len(x[1]), reverse=True)}
    tmp = []
    for i, (k, v) in tqdm(enumerate(component_groups.items())):
        for n, path in enumerate(v):
            tmp.append(path)
            is_update = True
            if n == 0:
                name = path.split('/')[-1]
                new_name = f"{i+1}_largest_dup_{name}"
                new_path = os.path.join(working_dir, name)
                shutil.move(path, new_path)
            else:
                name = path.split('/')[-1]
                new_name = f"{i+1}_largest_dup_{name}"
                new_path = os.path.join(trash_dir, new_name)
                shutil.move(path, new_path)
    unique_images = all_images.difference(similar_images)
    print(list(all_images)[0], list(similar_images)[0])
    for name in unique_images:
        new_name = f"unique_{name}"
        new_path = os.path.join(working_dir, name)
        shutil.move(os.path.join(working_dir, name), new_path)
    return is_update

/home/asi/camera/thamnt/tools/fastdup/.venv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Restart remove duplicate process

In [4]:
# !rm /home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/removed_dup/*
!rm /home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/remove_images/dup/*
!rm -r /home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/all_images_copy
!cp -r /home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/all_images /home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/all_images_copy

# !rm /home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/all_images/*
# !rm -r /home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/fastdup_results/*

/bin/bash: /bin/rm: Argument list too long


### Remove duplicated image in working directory 

In [8]:
working_dir = "/home/asi/camera/thamnt/dataset/vp_hho_calibration_data/hoi_so_copy"
trash_dir = "/home/asi/camera/thamnt/dataset/vp_hho_calibration_data/dup"
fastdup_dir = "/home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/fastdup_results"
fd = fastdup.create(input_dir=working_dir, work_dir=fastdup_dir)
fd.run(ccthreshold=0.85, overwrite=True)
connected_components_df , _ = fd.connected_components()
is_update = (len(connected_components_df) != 0)
while is_update:
    clusters_df = get_clusters(connected_components_df)
    remove_dup_image(working_dir, trash_dir, clusters_df)
    fd = fastdup.create(input_dir=working_dir, work_dir=fastdup_dir)
    fd.run(ccthreshold=0.85, overwrite=True)
    connected_components_df , _ = fd.connected_components()
    is_update = (len(connected_components_df) != 0)
    print("--------------")

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.
2024-05-16 10:08:47 [INFO] Found total 266922 images to run on, 266922 train, 0 test, name list 266922, counter 266922 
2024-05-16 10:15:26 [INFO] Found total 266922 images to run onmated: 0 Minutes
2024-05-16 10:15:48 [INFO] 22144) Finished write_index() NN model
2024-05-16 10:15:48 [INFO] Stored nn model index file /home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/fastdup_results/nnf.index
2024-05-16 10:16:00 [INFO] Total time took 432586 ms
2024-05-16 10:16:00 [INFO] Found a total of 46711 fully identical images (d>0.990), which are 8.75 % of total graph edges
2024-05-16 10:16:00 [INFO] Found a total of 132094 nearly identical images(d>0.980), which are 24.74 % of total graph edges
2024-05-16 10:16:00 [INFO] Found a total of 469266 above threshold images (d>0.900), which are 87.90 % of total graph edges
2024-05-16 10:16:00 [INFO] Found a total of 26693 outlier images         (d<0.050), whic

6850it [00:06, 1038.08it/s]


people_tracking_5_1fd34052-33d2-43f1-99ed-83c9c7d87dc1.jpg people_tracking_5_1fd34052-33d2-43f1-99ed-83c9c7d87dc1.jpg
FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.
2024-05-16 10:16:26 [INFO] Found total 9722 images to run on, 9722 train, 0 test, name list 9722, counter 9722 
2024-05-16 10:16:44 [INFO] Found total 9722 images to run ontimated: 0 Minutes
2024-05-16 10:16:44 [INFO] 302) Finished write_index() NN model
2024-05-16 10:16:44 [INFO] Stored nn model index file /home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/fastdup_results/nnf.index
2024-05-16 10:16:44 [INFO] Total time took 18480 ms
2024-05-16 10:16:44 [INFO] Found a total of 85 fully identical images (d>0.990), which are 0.44 % of total graph edges
2024-05-16 10:16:44 [INFO] Found a total of 455 nearly identical images(d>0.980), which are 2.34 % of total graph edges
2024-05-16 10:16:44 [INFO] Found a total of 7790 above threshold images (d>0.900), which are 40.06 % of total gr

484it [00:00, 3409.30it/s]


people_tracking_10_29fc3253-a942-416b-b619-f948a0e24ca6.jpg people_tracking_10_29fc3253-a942-416b-b619-f948a0e24ca6.jpg
FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.
2024-05-16 10:16:48 [INFO] Found total 3880 images to run on, 3880 train, 0 test, name list 3880, counter 3880 
2024-05-16 10:16:54 [INFO] Found total 3880 images to run ontimated: 0 Minutes
2024-05-16 10:16:54 [INFO] 282) Finished write_index() NN model
2024-05-16 10:16:54 [INFO] Stored nn model index file /home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/fastdup_results/nnf.index
2024-05-16 10:16:54 [INFO] Total time took 6374 ms
2024-05-16 10:16:54 [INFO] Found a total of 0 fully identical images (d>0.990), which are 0.00 % of total graph edges
2024-05-16 10:16:54 [INFO] Found a total of 0 nearly identical images(d>0.980), which are 0.00 % of total graph edges
2024-05-16 10:16:54 [INFO] Found a total of 27 above threshold images (d>0.900), which are 0.35 % of total graph e

19it [00:00, 10133.75it/s]

people_tracking_8_9889d8a6-aac4-4e1c-bdd9-7b3b50a67391.jpg people_tracking_10_0534fc2f-accf-4823-93c4-5b4e4e0afddc.jpg


FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.
2024-05-16 10:16:56 [INFO] Found total 3830 images to run on, 3830 train, 0 test, name list 3830, counter 3830 
2024-05-16 10:17:02 [INFO] Found total 3830 images to run ontimated: 0 Minutes
2024-05-16 10:17:03 [INFO] 150) Finished write_index() NN model
2024-05-16 10:17:03 [INFO] Stored nn model index file /home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/fastdup_results/nnf.index
2024-05-16 10:17:03 [INFO] Total time took 6223 ms
2024-05-16 10:17:03 [INFO] Found a total of 0 fully identical images (d>0.990), which are 0.00 % of total graph edges
2024-05-16 10:17:03 [INFO] Found a total of 0 nearly identical images(d>0.980), which are 0.00 % of total graph edges
2024-05-16 10:17:03 [INFO] Found a total of 0 above threshold images (d>0.900), which are 0.00 % of total graph edges
2024-05-16 10:17:03 [INFO] Found a total of 383 outlier images         (d<0.050), which are 5.00 % of total graph ed

/home/asi/camera/thamnt/tools/fastdup/.venv/lib/python3.7/site-packages/fastdup/fastdup_controller.py:432: UserWarning: No connected components found, try using a lower threshold
  warnings.warn(f'No connected components found, try using a lower threshold')


### Clustering image using dino model

##### Remove highly similar images

In [ ]:
import fastdup
fd = fastdup.create(input_dir="/home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/all_images",
                    work_dir="/home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/fastdup_results")
fd.run(model_path='/home/asi/camera/thamnt/clean_data_pipeline/large_models/dinov2_vits14.onnx', cc_threshold=0.7, d=384, nearest_neighbors_k=10, threshold=0.7, overwrite=True)

In [46]:
connected_components_df , _ = fd.connected_components()
connected_components_df

,index,component_id,sum,count,mean_distance,min_distance,max_distance,filename,error_code,is_valid,fd_index
0,7,7,36.9648,40.0,0.9241,0.8744,0.9655,all_images/05_1_15_16_18_108_pistol_0.9723253846168518_469.jpg,VALID,True,7
1,8,7,36.9648,40.0,0.9241,0.8744,0.9655,all_images/05_1_15_16_18_504_pistol_0.8298203349113464_469.jpg,VALID,True,8
2,9,7,36.9648,40.0,0.9241,0.8744,0.9655,all_images/05_1_15_16_20_016_pistol_0.9513394832611084_469.jpg,VALID,True,9
3,10,7,36.9648,40.0,0.9241,0.8744,0.9655,all_images/05_1_15_16_20_180_pistol_0.8597550988197327_469.jpg,VALID,True,10
4,11,7,36.9648,40.0,0.9241,0.8744,0.9655,all_images/05_1_15_16_20_434_pistol_0.9196553826332092_469.jpg,VALID,True,11
...,...,...,...,...,...,...,...,...,...,...,...
183,245,91,3.4003,4.0,0.8501,0.8501,0.8501,all_images/05_3_15_45_10_946_pistol_0.9958643913269043_566.jpg,VALID,True,245
184,246,91,3.4003,4.0,0.8501,0.8501,0.8501,all_images/05_3_15_45_26_769_pistol_0.9130306839942932_566.jpg,VALID,True,246
185,248,93,10.4448,12.0,0.8704,0.8543,0.8912,all_images/05_3_15_51_56_265_pistol_0.8509616255760193_596.jpg,VALID,True,248
186,249,93,10.4448,12.0,0.8704,0.8543,0.8912,all_images/05_3_15_52_00_470_pistol_0.9489467740058899_596.jpg,VALID,True,249


In [47]:
clusters_df = get_clusters(connected_components_df)
clusters_df.head(1)

,filename,mean_distance,count
component_id,,,
85,"[all_images/05_3_15_43_59_603_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_43_59_873_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_00_345_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_00_721_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_01_062_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_01_253_pistol_0.9904096126556396_546.jpg, all_images/05_3_15_44_01_487_pistol_0.9917787909507751_546.jpg, all_images/05_3_15_44_02_157_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_02_384_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_02_921_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_03_153_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_03_323_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_03_569_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_03_781_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_04_158_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_04_344_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_04_538_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_04_803_pistol_0.9942505359649658_546.jpg, all_images/05_3_15_44_04_977_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_05_207_pistol_0.9713682532310486_546.jpg, all_images/05_3_15_44_05_421_pistol_0.9904096126556396_546.jpg, all_images/05_3_15_44_05_723_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_05_988_pistol_0.9943296313285828_546.jpg, all_images/05_3_15_44_06_187_pistol_0.9913530945777893_546.jpg, all_images/05_3_15_44_06_564_pistol_0.9978783130645752_546.jpg, all_images/05_3_15_44_06_783_pistol_0.9891186952590942_546.jpg, all_images/05_3_15_44_06_982_pistol_0.9891186952590942_546.jpg, all_images/05_3_15_44_07_187_pistol_0.9982879757881165_546.jpg, all_images/05_3_15_44_07_439_pistol_0.9917787909507751_546.jpg, all_images/05_3_15_44_07_664_pistol_0.9917787909507751_546.jpg, all_images/05_3_15_44_07_856_pistol_0.9713682532310486_546.jpg]",0.979,31


In [48]:
from tqdm import tqdm
import shutil
import os

raw_dir = "/home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/all_images"
save_dir = "/home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/removed_dup"
remove_dup_dir = "/home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_vp_cleaning/remove_images/dup"

component_groups = {}
for i, r in clusters_df.iterrows():
    group_name = i
    filenames = r['filename']
    component_groups[group_name] = filenames
component_groups = {k:v for k, v in sorted(component_groups.items(), key=lambda x : len(x[1]), reverse=True)}
   
for i, (k, v) in tqdm(enumerate(component_groups.items())):
    for n, path in enumerate(v):
        if n == 0:
            name = path.split('/')[-1]
            new_name = f"{i+1}_highly_sim_{name}"
            new_path = os.path.join(save_dir, new_name)
            shutil.copy(path, new_path)
        else:
            name = path.split('/')[-1]
            new_name = f"{i+1}_highly_sim_{name}"
            new_path = os.path.join(remove_dup_dir, new_name)
            shutil.copy(path, new_path)
        
similar_images = set()
for i, r in clusters_df.iterrows():
    group_name = i
    filenames = r['filename']
    similar_images.update(filenames)
 
all_images = set([os.path.join(raw_dir, name) for name in os.listdir(raw_dir)])
unique_images = all_images.difference(similar_images)

for path in unique_images:
    name = path.split('/')[-1]
    new_name = f"unique_{name}"
    new_path = os.path.join(save_dir, new_name)
    shutil.copy(path, new_path)


31it [00:00, 2260.14it/s]


##### Cluster similar images


In [68]:
fd = fastdup.create(input_dir="3_data_raw_highly_similar_removed",
                    work_dir="analytics_results/fastdup")
fd.run(model_path='/home/asi/camera/thamnt/clean_data_pipeline/large_models/dinov2_vits14.onnx', cc_threshold=0.90, d=384, overwrite=True)

FastDup Software, (C) copyright 2022 Dr. Amir Alush and Dr. Danny Bickson.
2024-04-01 07:12:59 [INFO] Found resent/efficientnet/dinov2 model, setting up normalization
2024-04-01 07:12:59 [INFO] Going to loop over dir 3_data_raw_highly_similar_removed
2024-04-01 07:13:00 [INFO] Found total 2601 images to run on, 2601 train, 0 test, name list 2601, counter 2601 
2024-04-01 07:14:26 [INFO] Found total 2601 images to run ontimated: 0 Minutes
2024-04-01 07:14:26 [INFO] 81) Finished write_index() NN model
2024-04-01 07:14:26 [INFO] Stored nn model index file analytics_results/fastdup/nnf.index
2024-04-01 07:14:26 [INFO] Total time took 86233 ms
2024-04-01 07:14:26 [INFO] Found a total of 0 fully identical images (d>0.990), which are 0.00 % of total graph edges
2024-04-01 07:14:26 [INFO] Found a total of 0 nearly identical images(d>0.980), which are 0.00 % of total graph edges
2024-04-01 07:14:26 [INFO] Found a total of 1386 above threshold images (d>0.900), which are 26.64 % of total graph e

0

In [ ]:
connected_components_df , _ = fd.connected_components()
clusters_df = get_clusters(connected_components_df)

In [ ]:
from tqdm import tqdm
import shutil
import os

component_groups = {}
for i, r in clusters_df.iterrows():
    group_name = i
    filenames = r['filename']
    component_groups[group_name] = filenames
component_groups = {k:v for k, v in sorted(component_groups.items(), key=lambda x : len(x[1]), reverse=True)}
all_dir = "/home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_clean2_analysis/clean_dup/others_train"
save_dir = "/home/asi/camera/thamnt/dataset/weapon_cls/cleaning/data_clean2_analysis/components/others"
for i, (k, v) in tqdm(enumerate(component_groups.items())):
    for n, path in enumerate(v):
        name = path.split('/')[-1]
        new_name = f"{i+1}_comp_{name}"
        new_path = os.path.join(save_dir, new_name)
        shutil.copy(path, new_path)
        
similar_images = set()
for i, r in clusters_df.iterrows():
    group_name = i
    filenames = r['filename']
    similar_images.update(filenames)

all_images = set([os.path.join(all_dir, name) for name in os.listdir(all_dir)])
unique_images = all_images.difference(similar_images)

for path in unique_images:
    name = path.split('/')[-1]
    new_name = f"unique_{name}"
    new_path = os.path.join(save_dir, new_name)
    shutil.copy(path, new_path)


217it [00:00, 1815.72it/s]


### Outlier

In [47]:
outlier_df = fd.outliers()
outlier_df.head()

,outlier,nearest,distance,filename_outlier,index_x,error_code_outlier,is_valid_outlier,fd_index_outlier,filename_nearest,index_y,error_code_nearest,is_valid_nearest,fd_index_nearest
0,288,217,0.222928,2_data_raw_duplicate_removed/360_largest_dup_Sung2_Screenshot 2024-03-27 102756.png,288,VALID,True,288,2_data_raw_duplicate_removed/297_largest_dup_Sung2_xy-m1911-sung-luc-dan-thach-1.jpg,217,VALID,True,217
1,735,2430,0.267731,2_data_raw_duplicate_removed/unique_Sung0_d12a4d73638070c83093871adaa37f30.jpg,735,VALID,True,735,2_data_raw_duplicate_removed/unique_Sung2_images (52).jpg,2430,VALID,True,2430
2,1481,1317,0.269335,2_data_raw_duplicate_removed/unique_Sung0_jpeg_images (352).jpg,1481,VALID,True,1481,2_data_raw_duplicate_removed/unique_Sung0_jpeg_i (911).jpg,1317,VALID,True,1317
3,2241,2618,0.289172,2_data_raw_duplicate_removed/unique_Sung1_NI4OMVcHgc_k3_6LiWfazwITlAk.png,2241,VALID,True,2241,2_data_raw_duplicate_removed/unique_Sung4_jpeg_th (134).jpg,2618,VALID,True,2618
4,66,59,0.292021,2_data_raw_duplicate_removed/160_largest_dup_Sung2_JPG_Capture4.jpg,66,VALID,True,66,2_data_raw_duplicate_removed/154_largest_dup_Sung2_JPG_Capture1.jpg,59,VALID,True,59


In [49]:
list_of_outliers = outlier_df[outlier_df.distance < 0.68].filename_outlier.tolist()
outlier_dir = "removed_images/outliers/"
for path in list_of_outliers:
    name = path.split("/")[-1]
    shutil.copy(path, os.path.join(outlier_dir, name))

### Remove Bright & Dark Images

In [ ]:
stats_df = fd.img_stats()
dark_images = stats_df[stats_df['mean'] < 20]  
list_of_dark_images = dark_images['filename'].to_list()
list_of_dark_images

dark_dir = "removed_images/dark_images/"
for path in list_of_dark_images:
    name = path.split("/")[-1]
    shutil.move(path, os.path.join(dark_dir, name))

In [ ]:
bright_images = stats_df[stats_df['mean'] > 220.5]
list_of_bright_images = bright_images['filename'].to_list()
print(list_of_bright_images)
bright_dir = "removed_images/bright_images/"
for path in list_of_bright_images:
    name = path.split("/")[-1]
    shutil.copy(path, os.path.join(bright_dir, name))